In [ ]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

In [ ]:
playlistURL = "https://www.youtube.com/watch?v=VwabEKgX94s&list=PLTks92J980S2Dbx2WaSDAT1h35vzb7iMU&ab_channel=smoBEEUniverse"
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(playlistURL)
time.sleep(2)

In [ ]:
html = driver.page_source
time.sleep(2)
driver.close()

In [ ]:
soup = bs(html, 'html.parser')

In [ ]:
[a.get('href').split('v=')[1].split('&')[0] for a in soup.find_all("a", class_="ytd-playlist-panel-video-renderer")]

In [ ]:
spans = [span.text for span in soup.find_all("span", class_="ytd-playlist-panel-video-renderer")]
titles = [row[11:-9] for row in spans if ' / ' in row]
dates = [title.split(' / ')[0] for title in titles]
locations = [title.split(' / ')[1] for title in titles]
birds = [title.split(' / ')[2].split(' (')[0].split('（')[0].split(' #')[0] for title in titles]

In [ ]:
dates

In [ ]:
locations

In [ ]:
birds